## <span style=color:blue>This notebook builds the function soy_is_here(year,lat,lon), which produces "True" if the 100m x 100m area around lon-lat was a soybean field in the given year.  (At least, according to the data of USDA NASS.)  </span>

## <span style=color:blue>Then we import the dictionary with the lon-lat sequences for each county, and for each year find the first 20 that are in soybean fields, and right 

<span style=color:blue>First step is to create a function that tests, given a year-lon-lat triple, whether there was a soy field at lat-lon during the given year.  This is based on checking files downloaded from https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php.  To understand the meaning of the pixel values, please see https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/meta.php and the files in there, e.g., https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_ia22.htm.  Among other things, you will see that the value of '5' corresponds to soybean fields </span>

In [1]:
# first, examining the structure of the files I downloaded

dir_main = '/Users/rick/AG-CODE--v03/CROPSCAPE/DATA-DOWNLOADS/'

# following the structure of the directory name and file from downloaded zip files, which are organized by year
def pathname_for_year(year):
    last_dir_name = f'{str(year)}_30m_cdls/'
    file_name = f'{str(year)}_30m_cdls.tif'
    return dir_main + last_dir_name + file_name

# test
print(pathname_for_year(2022))

/Users/rick/AG-CODE--v03/CROPSCAPE/DATA-DOWNLOADS/2022_30m_cdls/2022_30m_cdls.tif


<span style=color:blue>Now we inspect structure of the tif files.     </span>

#### <span style=color:blue>Note that the Coordinate Reference System (CRS) is EPSG: 5070 rather than EPSG:4326 (also basically equivalent to WGS84), which is the one we are often using.  BTW, the unit of measure for EPSG:5070 is 1 meter, and so the pixels in these tif files are approximately 30m x 30m.  (See https://epsg.io/5070-1252) </span>

In [2]:
import json
import subprocess

def pull_useful(ginfo):  # should give as input the result.stdout from calling gdalinfo -json
    useful = {}
    useful['band_count'] = len(ginfo['bands'])
    useful['cornerCoordinates'] = ginfo['cornerCoordinates']
    useful['proj:transform'] = ginfo['stac']['proj:transform']
    useful['size'] = ginfo['size']
    useful['bbox'] = ginfo['stac']['proj:projjson']['bbox']
    useful['espgEncoding'] = ginfo['stac']['proj:epsg']
    return useful

path_to_file = pathname_for_year(2008)
# path_to_file = pathname_for_year(2022)

gdalInfoReq = " ".join(["gdalinfo", "-json", path_to_file])
# print(gdalInfoReq[k])
result = subprocess.run([gdalInfoReq], shell=True, capture_output=True, text=True)
# print(result)
print()
# print(result.stdout)
print()
gdalInfo = json.loads(result.stdout)

useful = pull_useful(gdalInfo)
print(json.dumps(useful, indent=2, sort_keys=True))



{
  "band_count": 1,
  "bbox": {
    "east_longitude": -66.91,
    "north_latitude": 49.38,
    "south_latitude": 24.41,
    "west_longitude": -124.79
  },
  "cornerCoordinates": {
    "center": [
      -48930.0,
      1724760.0
    ],
    "lowerLeft": [
      -2356095.0,
      276915.0
    ],
    "lowerRight": [
      2258235.0,
      276915.0
    ],
    "upperLeft": [
      -2356095.0,
      3172605.0
    ],
    "upperRight": [
      2258235.0,
      3172605.0
    ]
  },
  "espgEncoding": 5070,
  "proj:transform": [
    -2356095.0,
    30.0,
    0.0,
    3172605.0,
    0.0,
    -30.0
  ],
  "size": [
    153811,
    96523
  ]
}


<span style=color:blue>Function to transform from EPSG:4326 to EPSG:5070.  The rasterio-based function we use below will take coordinates in EPSG:5010, since the tif files we are using here are in EPSG:5010.     </span>

In [7]:
from pyproj import Transformer

transformer = Transformer.from_crs('EPSG:4326', 'EPSG:5070')

def from_4326_to_5070(lon,lat):
    # I'm not sure why the role positions of lon-lat are different on input and output
    # but that is what my numerous small test runs showed to me
    new_lon,new_lat = transformer.transform(lat,lon)
    return new_lon, new_lat

# test on coordinates from central Iowa
old_lon = -92.8
old_lat = 42.7
print(from_4326_to_5070(old_lon,old_lat))
# (you can check this at https://epsg.io/transform)


(260567.23440705295, 2193569.032086156)


<span style=color:blue>Function that fetches a 3x3 square of pixel values from the given tif file.  The pixels in the tif file correspond to  30m x 30m, so we are looking at a rouhgly 100m x 100m area that is all or mostly soybean field </span>

<span style=color:blue>Note that in 2008 the target area was planted mainly with maize, but in 2022 it was planted with soybeans</span>

In [66]:
import rasterio

# expects lon-lat to be in EPSG:4326.  
# These are converted to EPSG:5070 inside the function
def get_coordinate_pixels(tiff_file, lon, lat):
    dataset = rasterio.open(tiff_file)
    lon_new,lat_new = from_4326_to_5070(lon,lat)
    # print(lon_new,lat_new)
    py, px = dataset.index(lon_new, lat_new)
    # print(py, px)
    # create 3px x 3px window centered on the lon-lat
    window = rasterio.windows.Window(px-1, py-1, 3, 3)
    clip = dataset.read(window=window)
    return clip

    
# test
old_lon = -92.8
old_lat = 42.7
path_to_file = pathname_for_year(2008)
print(get_coordinate_pixels(path_to_file,old_lon,old_lat))
print()
path_to_file = pathname_for_year(2022)
print(get_coordinate_pixels(path_to_file,old_lon,old_lat))

[[[  1   1   1]
  [  1   1   1]
  [  1   1 176]]]

[[[5 5 5]
  [5 5 5]
  [5 5 5]]]


<span style=color:blue>Also, a function that that tests whether all 9 spots in the 3x3 square have a given value.  (We are interested in "5", which is soy beans.)</span>

In [16]:
# land_use_val should be an integer; see, e.g., 
#     https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_ia22.htm
#     for mapping from values to meanings
def usage_is_here(year, lon, lat, land_use_val):
    path_to_file = pathname_for_year(year)
    arr = get_coordinate_pixels(path_to_file, lon, lat)
    out = True
    for i in range(0,3):
        for j in range(0,3):
            out = out & (arr[0][i][j] == land_use_val)
    return out

def soy_is_here(year, lon, lat):
    return usage_is_here(year,lon,lat,5)

old_lon = -92.8
old_lat = 42.7
print(soy_is_here(2008, old_lon, old_lat))
print(soy_is_here(2022, old_lon, old_lat))

    

False
True


### <span style=color:blue>Importing the dictionary with lon-lat sequences.  Also setting a second dict that will hold lists lon-lats that are in soybean fields.</span>

In [37]:
archive_dir = '/Users/rick/AG-CODE--v03/ML-ARCHIVES--v01/'
in_file = 'state_county__seq_of_lon_lats.json'

f = open(archive_dir + in_file)
dict = json.load(f)

print(dict.keys())



dict_keys(['ILLINOIS', 'INDIANA', 'IOWA', 'MISSOURI', 'NEBRASKA', 'OHIO'])


<span style=color:blue>Function that scans through one list of lon-lats and finds first set that are in soybean fields</span>

In [44]:
def gen_soy_lon_lats(year,state,county, count):
    list = dict[state][county]
    i = 0
    out_list = []
    for ll in list:
        if soy_is_here(year, ll[0], ll[1]):
            out_list += [ll]
            i += 1
        if i == 20:
            return out_list, []
    print(f'\nFor {str(year)}, {state}, {county}: \nFailed to find {str(count)} lon-lats that were in soybean fields. Found only {str(i)}.\n')
    short_fall_record = [year, state, county, i]
    return out_list, short_fall_record

list, short = gen_soy_lon_lats(2008, 'ILLINOIS', 'MASSAC', 20)
print(list)
print(short)
print()
list, short = gen_soy_lon_lats(2008, 'MISSOURI', 'DALLAS', 20)
print(list)
print(short)


[[-89.1984554, 36.7738721], [-88.9657046, 36.7313958], [-88.819793, 36.8675512], [-88.6325773, 36.7654643], [-88.9978016, 37.6782879], [-89.1806347, 36.8884787], [-89.0443054, 37.2258879], [-88.7746352, 37.2689221], [-88.5048973, 36.9437909], [-89.108685, 37.5181656], [-88.6325461, 36.754499], [-88.81078, 36.8124781], [-89.1819939, 36.787544], [-88.6579458, 36.9255517], [-88.2954249, 36.7502558], [-88.786367, 37.0459674], [-88.6205757, 36.7924507], [-88.9235313, 37.3041449], [-88.6585063, 37.6831332], [-88.7716353, 37.2425947]]
[]


For 2008, MISSOURI, DALLAS: 
Failed to find 20 lon-lats that were in soybean fields. Found only 2.

[[-93.4923439, 37.5926585], [-93.3564492, 37.4714198]]
[2008, 'MISSOURI', 'DALLAS', 2]


<span style=color:blue>Function that generates a fixed number of lon-lats in soybean fields for each year and each county. This took quite a while to run completely -- about 4 hours.    </span>

In [48]:
import datetime

working_dir = '/Users/rick/AG-CODE--v03/ML-WITH-NDVI/OUTPUTS/OUTPUT-v01/'
dict1_file = 'year_state_county_soy_seq.json'
short_list = 'year_state_county_shortfalls.json'

def gen_all_soy_lists(dict, count):
    dict1 = {}
    for year in range(2008,2023):
        dict1[year] = {}
        for key in dict.keys():
            dict1[year][key] = {}    
    print(dict1.keys())
    print(dict1[2013].keys())
    
    shortfall_list = []

    i = 0
    for year in dict1.keys():
        for state in dict.keys():
            for county in dict[state].keys():
                list, short = gen_soy_lon_lats(year, state, county, count)
                dict1[year][state][county] = list
                if short != []:
                    shortfall_list += [short]
                                
                i += 1
                if i % 20 == 0:
                    print(f'Have generated soybean lon-lat lists for {str(i)} year-county pairs')
                if i % 50 == 0:
                    with open(working_dir + dict1_file, 'w') as fp:
                        json.dump(dict1, fp)
                    with open(working_dir + short_list, 'w') as fp:
                        json.dump(shortfall_list, fp)
                
    return dict1, shortfall_list
                    
print(datetime.datetime.now())
dict1, short = gen_all_soy_lists(dict, 20)
print(datetime.datetime.now())
                

2023-05-28 22:20:00.031447
dict_keys([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])
dict_keys(['ILLINOIS', 'INDIANA', 'IOWA', 'MISSOURI', 'NEBRASKA', 'OHIO'])
Have generated soybean lon-lat lists for 20 year-county pairs
Have generated soybean lon-lat lists for 40 year-county pairs
Have generated soybean lon-lat lists for 60 year-county pairs
Have generated soybean lon-lat lists for 80 year-county pairs
Have generated soybean lon-lat lists for 100 year-county pairs
Have generated soybean lon-lat lists for 120 year-county pairs
Have generated soybean lon-lat lists for 140 year-county pairs
Have generated soybean lon-lat lists for 160 year-county pairs
Have generated soybean lon-lat lists for 180 year-county pairs
Have generated soybean lon-lat lists for 200 year-county pairs
Have generated soybean lon-lat lists for 220 year-county pairs
Have generated soybean lon-lat lists for 240 year-county pairs
Have generated soybean lon-lat lists for 260

Have generated soybean lon-lat lists for 1060 year-county pairs
Have generated soybean lon-lat lists for 1080 year-county pairs

For 2009, OHIO, BELMONT: 
Failed to find 20 lon-lats that were in soybean fields. Found only 6.

Have generated soybean lon-lat lists for 1100 year-county pairs

For 2009, OHIO, GALLIA: 
Failed to find 20 lon-lats that were in soybean fields. Found only 15.


For 2009, OHIO, GUERNSEY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 16.


For 2009, OHIO, MEIGS: 
Failed to find 20 lon-lats that were in soybean fields. Found only 9.


For 2009, OHIO, MORGAN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 15.


For 2009, OHIO, NOBLE: 
Failed to find 20 lon-lats that were in soybean fields. Found only 3.


For 2009, OHIO, WASHINGTON: 
Failed to find 20 lon-lats that were in soybean fields. Found only 4.

Have generated soybean lon-lat lists for 1120 year-county pairs
Have generated soybean lon-lat lists for 1140 year-county


For 2011, NEBRASKA, SHERIDAN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2011, NEBRASKA, CHERRY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.

Have generated soybean lon-lat lists for 2080 year-county pairs
Have generated soybean lon-lat lists for 2100 year-county pairs
Have generated soybean lon-lat lists for 2120 year-county pairs

For 2011, NEBRASKA, DUNDY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 14.

Have generated soybean lon-lat lists for 2140 year-county pairs
Have generated soybean lon-lat lists for 2160 year-county pairs
Have generated soybean lon-lat lists for 2180 year-county pairs
Have generated soybean lon-lat lists for 2200 year-county pairs

For 2011, OHIO, BELMONT: 
Failed to find 20 lon-lats that were in soybean fields. Found only 5.


For 2011, OHIO, JEFFERSON: 
Failed to find 20 lon-lats that were in soybean fields. Found only 15.

Have generated soybean lon-lat lists for 2220 ye


For 2013, NEBRASKA, CHEYENNE: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2013, NEBRASKA, DEUEL: 
Failed to find 20 lon-lats that were in soybean fields. Found only 10.


For 2013, NEBRASKA, GARDEN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2013, NEBRASKA, SHERIDAN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2013, NEBRASKA, CHERRY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.

Have generated soybean lon-lat lists for 3200 year-county pairs
Have generated soybean lon-lat lists for 3220 year-county pairs
Have generated soybean lon-lat lists for 3240 year-county pairs

For 2013, NEBRASKA, DUNDY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 18.

Have generated soybean lon-lat lists for 3260 year-county pairs
Have generated soybean lon-lat lists for 3280 year-county pairs
Have generated soybean lon-lat lists for 3300 year-county pairs
Have g


For 2015, MISSOURI, WEBSTER: 
Failed to find 20 lon-lats that were in soybean fields. Found only 2.

Have generated soybean lon-lat lists for 4300 year-county pairs

For 2015, NEBRASKA, CHEYENNE: 
Failed to find 20 lon-lats that were in soybean fields. Found only 1.


For 2015, NEBRASKA, DEUEL: 
Failed to find 20 lon-lats that were in soybean fields. Found only 11.


For 2015, NEBRASKA, GARDEN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2015, NEBRASKA, SHERIDAN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 0.


For 2015, NEBRASKA, CHERRY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 1.

Have generated soybean lon-lat lists for 4320 year-county pairs
Have generated soybean lon-lat lists for 4340 year-county pairs

For 2015, NEBRASKA, DUNDY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 15.

Have generated soybean lon-lat lists for 4360 year-county pairs
Have generated soybean lon-lat l


For 2017, NEBRASKA, CHEYENNE: 
Failed to find 20 lon-lats that were in soybean fields. Found only 3.


For 2017, NEBRASKA, GARDEN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 5.


For 2017, NEBRASKA, SHERIDAN: 
Failed to find 20 lon-lats that were in soybean fields. Found only 1.


For 2017, NEBRASKA, CHERRY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 1.

Have generated soybean lon-lat lists for 5440 year-county pairs
Have generated soybean lon-lat lists for 5460 year-county pairs
Have generated soybean lon-lat lists for 5480 year-county pairs
Have generated soybean lon-lat lists for 5500 year-county pairs
Have generated soybean lon-lat lists for 5520 year-county pairs
Have generated soybean lon-lat lists for 5540 year-county pairs

For 2017, OHIO, BELMONT: 
Failed to find 20 lon-lats that were in soybean fields. Found only 7.

Have generated soybean lon-lat lists for 5560 year-county pairs
Have generated soybean lon-lat lists for 5580 


For 2019, NEBRASKA, CHERRY: 
Failed to find 20 lon-lats that were in soybean fields. Found only 3.

Have generated soybean lon-lat lists for 6560 year-county pairs
Have generated soybean lon-lat lists for 6580 year-county pairs
Have generated soybean lon-lat lists for 6600 year-county pairs
Have generated soybean lon-lat lists for 6620 year-county pairs
Have generated soybean lon-lat lists for 6640 year-county pairs
Have generated soybean lon-lat lists for 6660 year-county pairs

For 2019, OHIO, BELMONT: 
Failed to find 20 lon-lats that were in soybean fields. Found only 5.

Have generated soybean lon-lat lists for 6680 year-county pairs

For 2019, OHIO, JEFFERSON: 
Failed to find 20 lon-lats that were in soybean fields. Found only 16.

Have generated soybean lon-lat lists for 6700 year-county pairs

For 2019, OHIO, MEIGS: 
Failed to find 20 lon-lats that were in soybean fields. Found only 18.


For 2019, OHIO, NOBLE: 
Failed to find 20 lon-lats that were in soybean fields. Found only


For 2021, OHIO, BELMONT: 
Failed to find 20 lon-lats that were in soybean fields. Found only 7.

Have generated soybean lon-lat lists for 7800 year-county pairs

For 2021, OHIO, GALLIA: 
Failed to find 20 lon-lats that were in soybean fields. Found only 18.


For 2021, OHIO, MEIGS: 
Failed to find 20 lon-lats that were in soybean fields. Found only 13.

Have generated soybean lon-lat lists for 7820 year-county pairs

For 2021, OHIO, NOBLE: 
Failed to find 20 lon-lats that were in soybean fields. Found only 8.


For 2021, OHIO, WASHINGTON: 
Failed to find 20 lon-lats that were in soybean fields. Found only 4.

Have generated soybean lon-lat lists for 7840 year-county pairs
Have generated soybean lon-lat lists for 7860 year-county pairs
Have generated soybean lon-lat lists for 7880 year-county pairs
Have generated soybean lon-lat lists for 7900 year-county pairs
Have generated soybean lon-lat lists for 7920 year-county pairs
Have generated soybean lon-lat lists for 7940 year-county pair

<span style=color:blue>Save the dict1 and also the shortfalls    </span>

In [51]:
archive_dir = '/Users/rick/AG-CODE--v03/ML-ARCHIVES--v01/'
dict1_file = 'year_state_county_soy_seq.json'
short_list = 'year_state_county_shortfalls.json'

with open(archive_dir + dict1_file, 'w') as fp:
    json.dump(dict1, fp)
with open(archive_dir + short_list, 'w') as fp:
    json.dump(short, fp)


<span style=color:blue>Collecting year-state-county with zero hits </span>

In [55]:
zero_falls = []

for l in short:
    if l[3] == 0:
        zero_falls += [[l]]
        
print(len(zero_falls))

print(json.dumps(zero_falls,indent=4))

archive_dir = '/Users/rick/AG-CODE--v03/ML-ARCHIVES--v01/'
zero_file = 'year_state_county_soy_zero_falls.json'
with open(archive_dir + zero_file, 'w') as fp:
    json.dump(zero_falls, fp)



32
[
    [
        [
            2008,
            "MISSOURI",
            "CHRISTIAN",
            0
        ]
    ],
    [
        [
            2008,
            "MISSOURI",
            "WEBSTER",
            0
        ]
    ],
    [
        [
            2008,
            "NEBRASKA",
            "GARDEN",
            0
        ]
    ],
    [
        [
            2008,
            "NEBRASKA",
            "SHERIDAN",
            0
        ]
    ],
    [
        [
            2008,
            "OHIO",
            "WASHINGTON",
            0
        ]
    ],
    [
        [
            2009,
            "NEBRASKA",
            "CHEYENNE",
            0
        ]
    ],
    [
        [
            2009,
            "NEBRASKA",
            "SHERIDAN",
            0
        ]
    ],
    [
        [
            2010,
            "NEBRASKA",
            "SHERIDAN",
            0
        ]
    ],
    [
        [
            2011,
            "MISSOURI",
            "CHRISTIAN",
            

<span style=color:blue>Checking if any year-state-county in zero_falls had a positive yield in year_state_county_yield table</span>

In [71]:
import pandas as pd

archive_dir = '/Users/rick/AG-CODE--v03/ML-ARCHIVES--v01/'
yscy_file = 'year_state_county_yield.csv'

df_yscy = pd.read_csv(archive_dir + yscy_file)
print('Top of df_yscy')
print(df_yscy.head())

zero_with_yield = []
for l in zero_falls:
    year = l[0][0]
    state = l[0][1]
    county = l[0][2]
    rows = df_yscy[(df_yscy['year'] == year) & \
                  (df_yscy['state_name'] == state) & \
                  (df_yscy['county_name'] == county)]
    if len(rows) > 0:
        y = rows['yield'].iloc[0]
        zero_with_yield += [{'year':year, 'state_name':state, \
                             'county_name':county, 'yield':y}]

print('\nLength of zero_with_yield is: ', len(zero_with_yield))
print('\nListing of zero_with_yield')
df_zwy = pd.DataFrame(zero_with_yield)
print(df_zwy.head(30))
    
zero_with_yield = 'year_state_county_soy_zero_with_yield.csv'
df_zwy.to_csv(archive_dir + zero_with_yield, index=False)


Top of df_yscy
   year state_name county_name  yield
0  2022   ILLINOIS      BUREAU   67.5
1  2021   ILLINOIS      BUREAU   66.4
2  2020   ILLINOIS      BUREAU   64.8
3  2019   ILLINOIS      BUREAU   57.4
4  2018   ILLINOIS      BUREAU   68.5

Length of zero_with_yield is:  8

Listing of zero_with_yield
   year state_name county_name  yield
0  2008   MISSOURI     WEBSTER   43.0
1  2008       OHIO  WASHINGTON   36.5
2  2011   NEBRASKA    CHEYENNE   40.4
3  2013   NEBRASKA    CHEYENNE   38.6
4  2013   NEBRASKA      GARDEN   55.3
5  2014   MISSOURI   CHRISTIAN   30.3
6  2014   NEBRASKA      GARDEN   56.3
7  2020   MISSOURI   CHRISTIAN   34.4


<span style=color:blue>For this exercise, we will drop these year-state-county triples from consideration.  A more thorough approach would be to focus on these year-state-county pairs (and perhaps the other ones with < 20 lon-lats), and randomly generate more lon-lats within the county until at least a few are found inside soybean fields.  (On the one hand, there have to be some if there was a yield ... however, CropScape is not perfect and may not have identified them accurately.)</span>